In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept

from loading_data import get_data
from utils_maze import get_bin_centers, get_subset_zones, get_xy_idx

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "trials")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:
import info.r063d7 as r063d7
infos = [r063d7]
from run import analysis_infos
# infos = analysis_infos

In [ ]:
def trials_by_trajectory(info, sliced_position, zone, min_epoch=1.5, min_distance=20., merge_gap=1.5):
    x_idxs, y_idxs = get_xy_idx(info, sliced_position)

    in_zone = zone[y_idxs, x_idxs]
    jumps = np.diff(in_zone.astype(int))
    jumps = np.insert(jumps,0,0)

    starts = sliced_position.time[jumps == 1]
    np.insert(starts, 0, sliced_position.time[0])
    stops = sliced_position.time[jumps == -1]
    np.insert(stops, 0, sliced_position.time[-1])

    zone_epochs = nept.Epoch([starts, stops]).merge(gap=merge_gap)

    dur_idx = zone_epochs.durations >= min_epoch
    
    start_idxs = [nept.find_nearest_idx(sliced_position.time, start) for start in zone_epochs.starts]
    stop_idxs = [nept.find_nearest_idx(sliced_position.time, stop) for stop in zone_epochs.stops]

    dist_idx = np.zeros(zone_epochs.n_epochs).astype(bool)
    for i, (start_idx, stop_idx) in enumerate(zip(start_idxs, stop_idxs)):
        dist_idx[i] = sliced_position[start_idx].distance(sliced_position[stop_idx])[0] > min_distance

    return nept.Epoch([zone_epochs.starts[dur_idx&dist_idx], zone_epochs.stops[dur_idx&dist_idx]])

In [ ]:
def find_matched_trials(fewest_epochs, trials_to_match):
    starts = []
    stops = []
    centers = trials_to_match.centers
    for trial_center in fewest_epochs.centers:
        idx = np.nanargmin(np.abs(centers - trial_center))
        starts.append(trials_to_match[idx].start)
        stops.append(trials_to_match[idx].stop)
        centers[idx] = np.nan
    return nept.Epoch([starts, stops])

In [ ]:
def get_matched_trials(info, sliced_position):
    u_zone, shortcut_zone, novel_zone = get_subset_zones(info, sliced_position)
    
    
    
    u_epochs = trials_by_trajectory(info, sliced_position, u_zone)
    shortcut_epochs = trials_by_trajectory(info, sliced_position, shortcut_zone)
    novel_epochs = trials_by_trajectory(info, sliced_position, novel_zone, min_distance=0.)
    trial_epochs = [u_epochs, shortcut_epochs, novel_epochs]

    fewest_trials = trial_epochs[np.argmin([epoch.n_epochs for epoch in trial_epochs])]
    
    matched_trials = nept.Epoch([], [])
    for trial_epoch in trial_epochs:
        matched_trials = matched_trials.join(find_matched_trials(fewest_trials, trial_epoch))
        
    return matched_trials

In [ ]:
for info in infos:
    events, position, spikes, lfp, _ = get_data(info)
    sliced_position = position.time_slice(info.task_times["phase3"].start, info.task_times["phase3"].stop)
    matched_trials = get_matched_trials(info, sliced_position)
    print(info.session_id, matched_trials.n_epochs)
    
    for i, (trial_start, trial_stop) in enumerate(zip(matched_trials.starts, matched_trials.stops)):
        tt = sliced_position.time_slice(trial_start, trial_stop)

        plt.plot(position.x, position.y, "y.")
        plt.plot(tt.x, tt.y, "k.", ms=4)
        plt.savefig(os.path.join(output_filepath, info.session_id+"_trial-"+str(i)+".png"))

In [ ]:
matched_trials.n_epochs

In [ ]:
for trial_start, trial_stop in zip(matched_trials.starts, matched_trials.stops):
    print(trial_start, trial_stop)
    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
events, position, spikes, lfp, _ = get_data(info)

u_zone, shortcut_zone, novel_zone = get_subset_zones(info, position)

sliced_position = position.time_slice(info.task_times["phase3"].start, info.task_times["phase3"].stop)

In [ ]:
u_epochs = trials_by_trajectory(info, sliced_position, u_zone)
shortcut_epochs = trials_by_trajectory(info, sliced_position, shortcut_zone)
novel_epochs = trials_by_trajectory(info, sliced_position, novel_zone, min_distance=0.)
trial_epochs = [u_epochs, shortcut_epochs, novel_epochs]

In [ ]:
fewest_trials = trial_epochs[np.argmin([epoch.n_epochs for epoch in trial_epochs])]
n_trials = fewest_trials.n_epochs

In [ ]:
matched_trials = nept.Epoch([], [])
for trial_epoch in trial_epochs:
    matched_trials = matched_trials.join(find_matched_trials(fewest_trials, trial_epoch))

In [ ]:
for trial_start, trial_stop in zip(matched_trials.starts, matched_trials.stops):
    print(trial_start, trial_stop)
    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
n_trials

In [ ]:
for zone in [u_zone, shortcut_zone]:
    zone_epochs = trials_by_trajectory(info, sliced_position, zone)
    print(zone_epochs.n_epochs)
    tt = sliced_position.time_slice(zone_epochs.starts, zone_epochs.stops)

    plt.plot(position.x, position.y, "c.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()
    
    for trial_start, trial_stop in zip(zone_epochs.starts, zone_epochs.stops):
        print(trial_start, trial_stop)
        tt = sliced_position.time_slice(trial_start, trial_stop)

        plt.plot(position.x, position.y, "y.")
        plt.plot(tt.x, tt.y, "k.", ms=4)
        plt.show()
        
# Novel trajectory called separately because trial entry and exit are from the same location
zone_epochs = trials_by_trajectory(info, sliced_position, novel_zone, min_distance=0.)
print(zone_epochs.n_epochs)
tt = sliced_position.time_slice(zone_epochs.starts, zone_epochs.stops)

plt.plot(position.x, position.y, "c.")
plt.plot(tt.x, tt.y, "k.", ms=4)
plt.show()

for trial_start, trial_stop in zip(zone_epochs.starts, zone_epochs.stops):
    print(trial_start, trial_stop)
    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
matched_trials.n_epochs / n_trials

In [ ]:
uu

In [ ]:
matched_u = matched_trials(shortcut_epochs, u_epochs)
matched_shortcut = matched_trials(shortcut_epochs, shortcut_epochs)
matched_novel = matched_trials(shortcut_epochs, novel_epochs)

In [ ]:
for trial_start, trial_stop in zip(matched_u.starts, matched_u.stops):
    print(trial_start, trial_stop)
    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
for trial_start, trial_stop in zip(matched_shortcut.starts, matched_shortcut.stops):
    print(trial_start, trial_stop)
    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
for trial_start, trial_stop in zip(matched_novel.starts, matched_novel.stops):
    print(trial_start, trial_stop)
    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
s = 8613.776451  
e = 8614.877779999999

whatsupwiththistrial = position.time_slice(s-10, e+10)
plt.plot(position.x, position.y, "m.")
plt.plot(whatsupwiththistrial.x, whatsupwiththistrial.y, "k.")
plt.show()

In [ ]:
xx, yy = np.meshgrid(info.xedges, info.yedges)

plt.figure()
pp = plt.pcolormesh(xx, yy, u_zone, cmap="PuBuGn")
plt.plot(position.x, position.y, "k.", ms=2)
plt.tight_layout()
plt.show()

In [ ]:
from utils_maze import get_trials

In [ ]:
phase = info.task_times["phase3"]
trials = get_trials(events, phase)

for trial_start, trial_stop in zip(trials.starts, trials.stops):

    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
trials.n_epochs